**Mounting a Google Drive**

> Add blockquote



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Path to your CSV file
csv_file_path = '/content/drive/MyDrive/HateSpeechDataset.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)
df.head()

,Content,Label,Content_int
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22..."
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,..."
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,..."
4,afro american basho,1,"[146715, 46, 47, 48, 146714]"


In [ ]:
train = df.drop(['Content_int'], axis = 1)

In [ ]:
import nltk
import string
import re
from nltk.corpus import stopwords

# Download the 'stopwords' resource
nltk.download('stopwords')

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)



#Removing the stopwords from text
def split_into_words(text):
    # split into words by white space
    words = text.split()
    return words

def to_lower_case(words):
    # convert to lower case
    words = [word.lower() for word in words]
    return words

def remove_punctuation(words):
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    stripped = [re_punc.sub('', w) for w in words]
    return stripped

def keep_alphabetic(words):
    # remove remaining tokens that are not alphabetic
    words = [word for word in words if word.isalpha()]
    return words

def remove_stopwords(words):
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

def to_sentence(words):
    # join words to a sentence
    return ' '.join(words)
def tweet(words):
    tweet_tokenizer = nltk.tokenize.TweetTokenizer(strip_handles=True,reduce_len=True)
    tweet = tweet_tokenizer.tokenize(words)
    return tweet

#Removing the noisy text
def denoise_text(text):
    words = split_into_words(text)
    words = to_lower_case(words)
    words = remove_punctuation(words)
    words = keep_alphabetic(words)
    words = remove_stopwords(words)
    return to_sentence(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train['Content']=train['Content'].apply(denoise_text)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder

X = train.Content
Y = train.Label
le = LabelEncoder()  # Now you can use LabelEncoder
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences # Import pad_sequences

max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len) # Use pad_sequences directly

In [ ]:
pip install transformers tensorflow


In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer

def build_bert_model(max_len, bert_model_name='bert-base-uncased'):
    # Load pre-trained BERT model and tokenizer
    bert_model = TFBertModel.from_pretrained(bert_model_name)
    tokenizer = BertTokenizer.from_pretrained(bert_model_name)

    # Define inputs
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    # BERT encoder
    bert_output = bert_model(input_ids, attention_mask=attention_mask)
    sequence_output = bert_output.last_hidden_state

    # Adding custom layers on top of BERT
    x = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(sequence_output)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Define the model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

    return model, tokenizer

# Example usage
max_len = 128  # Adjust as per your requirement
model, tokenizer = build_bert_model(max_len)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                       

In [ ]:
!pip install datasets pandas transformers tensorflow
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from datasets import load_dataset # Import the load_dataset function
import pandas as pd # Import pandas to handle CSV files
from sklearn.model_selection import train_test_split




def encode_data(texts, tokenizer, max_len):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='tf',
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

def build_bert_model(max_len, bert_model_name='bert-base-uncased'):
    # Load pre-trained BERT model and tokenizer
    bert_model = TFBertModel.from_pretrained(bert_model_name)
    tokenizer = BertTokenizer.from_pretrained(bert_model_name)

    # Define inputs
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    # BERT encoder
    bert_output = bert_model(input_ids, attention_mask=attention_mask)
    sequence_output = bert_output.last_hidden_state

    # Adding custom layers on top of BERT
    x = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(sequence_output)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Define the model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

    return model, tokenizer

# Load and preprocess the dataset
# Assuming your dataset is a CSV file, use pandas to load it
dataset_path = "/content/drive/MyDrive/HateSpeechDataset.csv" # Update with the correct path
df = pd.read_csv(dataset_path)

# Split the dataset into train and test sets (adjust test_size as needed)
train_df, test_df = train_test_split(df, test_size=0.2)

max_len = 128  # Adjust as per your requirement
batch_size = 32

train_texts = train_df['Content'].tolist() # Assuming 'text' is the column containing text data
train_labels = train_df['Label'].tolist() # Assuming 'label' is the column containing labels
test_texts = test_df['Content'].tolist()
test_labels = test_df['Label'].tolist()

# Convert string labels to numerical labels (assuming '0' and '1' are the string labels)
train_labels = [0 if label == '0' else 1 for label in train_labels]
test_labels = [0 if label == '0' else 1 for label in test_labels]

model, tokenizer = build_bert_model(max_len)

train_input_ids, train_attention_masks = encode_data(train_texts, tokenizer, max_len)
test_input_ids, test_attention_masks = encode_data(test_texts, tokenizer, max_len)

train_labels = tf.convert_to_tensor(train_labels)
test_labels = tf.convert_to_tensor(test_labels)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    [train_input_ids, train_attention_masks], train_labels,
    validation_data=([test_input_ids, test_attention_masks], test_labels),
    epochs=3,
    batch_size=batch_size
)

# Evaluate the model
results = model.evaluate([test_input_ids, test_attention_masks], test_labels)

print(results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture

Epoch 1/3


  185/11023 [..............................] - ETA: 148:47:42 - loss: 0.3822 - accuracy: 0.8258